## FDW Crop Production Data Profiling - Malawi

In [1]:
import glob, json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools import FDW_PD_CaliSeasonYear
from tools_graphic import PlotBarProduction, PlotLinePAY
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [2]:
# CPCV2 grain code ------------------------------ #
grain_code = pd.read_hdf('../data/crop/grain_cpcv2_code.hdf')
product_category = grain_code[['product', 'product_category']].set_index('product').to_dict()['product_category']
# ----------------------------------------------- #

# Load FEWS NET administrative boundaries ------- #
epsg = 'EPSG:32736' # Malawi
fn_shapes = sorted(glob.glob('../data/shapefile/fewsnet/MW_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
auth = tuple(json.loads(open('token.json', "r").read()))
parameters = {
    'format': 'json',
    'country': 'Malawi',
    'product': 'R011',
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
df_origin = df.copy()
# ----------------------------------------------- #

- FEWS NET admin shapefiles ------------------- #
| year	 | Admin1   | # units   | Admin2   | # units   | Admin3   | # units   |
| 1971	 | MW1971A1 | 3	| MW1971A2	| 24	| nan	| 0	|
| 1998	 | MW1998A1 | 3	| MW1998A2	| 27	| nan	| 0	|
| 2003	 | MW2003A1 | 3	| MW2003A2	| 28	| nan	| 0	|
| 2018	 | MW2018A1 | 3	| MW2018A2	| 32	| nan	| 0	|
----------------------------------------------- #


In [3]:
# Manual Pre-processing before Sweeping --------- #
# 1. Default setting 
# a) None-type crop production system
df.loc[df['crop_production_system'].isna(), 'crop_production_system'] = 'none'
df.loc[df['crop_production_system'] == '', 'crop_production_system'] = 'none'
# b) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'
# 2. Manual setting
# a) Add admin names to reporting units
admin_level = 'admin_3'
reporting_unit = df.loc[df['fnid'].map(lambda x: x[6] != 'A'), 'fnid'].unique()
df.loc[df.fnid.isin(reporting_unit),admin_level] = df.loc[df.fnid.isin(reporting_unit), 'locality_name'].map(lambda x: x.split(' ')[0])
# ----------------------------------------------- #

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df, area_priority="Area Planted")
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([MW_Admin2_1971, MW_Admin2_1998, MW_Admin2_2003], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #

- Remove missing records ---------------------- #
Orignial data points: 11,472
Removed 600 "Missing Value" points
3,744/3,824 "Area Planted" points are retained.
3,564/3,824 "Quantity Produced" points are retained.
3,564/3,824 "Yield" points are retained.
Current data points: 10,872

- Minor changes are applied ------------------- #

- Basic information --------------------------- #
Data period: 1982 - 2019
7 grain types are found: Maize Grain (White), Millet, Millet (Finger), Millet (Pearl), Rice (Paddy), Sorghum, Wheat Grain
2 seasons are found: Winter (09-01), Main (04-01)
3 crop production system are found: none, estate, Estate (PS)
Data sources include:
[1] Ministry of Agriculture, Irrigation and Water Development, Malawi --- Annual Production Estimate Survey, Malawi
[2] Ministry of Agriculture, Irrigation and Water Development, Malawi --- Annual Production Estimate Survey, Malawi,
[3] Ministry of Agriculture, Irrigation and Water Development, Malawi --- FEWS NET Agro-Maps, Malawi


- Malawi crop seasonal calendar

![FEWS NET](https://fews.net/sites/default/files/styles/large_width_880/public/2023-03/seasonal-calendar-malawi.png?itok=Vq_GZunP)

![image](../figures/crop_calendar/fao_malawi_crop_calendar.jpeg)

- FDW data consists of `MW1971A2`, `MW1998A2`, and `MW2003A2`.

| Year | Admin-1 | # units  | Admin-2  | # units |
| :---: | :----:  | :----:   | :----:   | :---:  |
| 1971 | MW1971A1 | 3 | **`MW1971A2`** | 24 |
| 1998 | MW1998A1 | 3 | **`MW1998A2`** | 27 |
| 2003 | MW2003A1 | 3 | **`MW2003A2`** | 28 |
| 2018 | MW2018A1 | 3 | MW2018A2 | 32 |

- Comparison between admin boundaries.

![image](https://github.com/chc-ucsb/gscd/blob/main/figures/MW_admin_shapes.png?raw=true)

- In 1998, 3 districts are divided and added.

| 1971-1998 (original) |1998-2003 (changed)|1998-2003 (added)|
| :---:| :---:|:---:|
|Nkhata Bay (MW1971A20107)    | Nkhata Bay (MW1998A20103)    | Likoma (MW1998A20106)|
|Mulanje (MW1971A20315)       | Mulanje (MW1998A20308)       | Phalombe (MW1998A20309)|
|Machinga (MW1971A20316)      | Machinga (MW1998A20302)      | Balaka (MW1998A20312)|

 ** Nkhata Bay (MW1971A20107) is divided into a very big district (MW1998A20103, Nkhata Bay) with a very small island (MW1998A20106, Likoma).

- In 2003, 1 district is divided and added.

| 1998-2003 (original) |2003-current (changed)|2003-current (added)|
| :---:| :---:|:---:|
|Mwanza (MW1998A20314)    | Mwanza (MW2003A20306)    | Neno (MW2003A20313)|

- Four reporting units exist:

| Admin-2 | Reporting units |
| :---: | :---: |
| MW2003A20105 (Mzimba)|  MW2012R3010501 (Mzimba North RDP), MW2012R3010502 (Mzimba South RDP) |
| MW2003A20206 (Lilongwe)|  MW2014R3020601 (Lilongwe West RDP), MW2014R3020602 (Lilongwe East RDP) |

- **`MW2003A2`** is used to represent current admin-level 2 crop data.
- Malawi has two crop seasons: `Main` and `Winter` (from 2006).
- Malawi has three crop production systems: `None`, `Estate (PS)`, and `estate`.

In [4]:
# Define the latest shapefile ------------------- #
latest_level = 2
shape_latest = MW_Admin2_2003.copy().to_crs('epsg:4326')
# ----------------------------------------------- #

# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
# ----------------------------------------------- #

# FDW Production Data Compiling ----------------- #
area, prod = FDW_PD_Compiling(df, shape_used)
area_all, prod_all = area.copy(), prod.copy()
# - Aggregate grain data by grain type
FlagGrainType = True
[area, prod, area_all, prod_all] = FDW_PD_GrainTypeAgg([area, prod, area_all, prod_all], product_category)
mdx_pss = area.columns.droplevel([0,1]).unique()
# ----------------------------------------------- #

- Aggregation of grain types ------------------ #
7 crops: Maize Grain (White), Millet, Millet (Finger), Millet (Pearl), Rice (Paddy), Sorghum, Wheat Grain
5 crops: Maize, Millet, Rice, Sorghum, Wheat



In [5]:
# Link admin boundaries ------------------------- #
link_1971, over_1971 = FDW_PD_CreateAdminLink(MW_Admin2_1971, MW_Admin2_2003, 'ADMIN2', 'ADMIN2', prod, epsg)
assert all(np.unique([v['method'] for k,v in link_1971.items()]) == 'PBR')
link_1998, over_1998 = FDW_PD_CreateAdminLink(MW_Admin2_1998, MW_Admin2_2003, 'ADMIN2', 'ADMIN2', prod, epsg)
assert all(np.unique([v['method'] for k,v in link_1998.items()]) == 'PBR')
# Crop specific ratios
link_ratio_1971 = FDW_PD_RatioAdminLink(link_1971, prod, over_1971, mdx_pss)
link_ratio_1998 = FDW_PD_RatioAdminLink(link_1998, prod, over_1998, mdx_pss)
# Merge link_ratio
assert link_ratio_1971.keys() == link_ratio_1998.keys()
link_merged = [link_ratio_1971, link_ratio_1998]
fnids_new = list(link_merged[0].keys())
link_ratio = dict()
for fnid in fnids_new:
    container = []
    for link in link_merged:
        container.append(link[fnid])
    link_ratio[fnid] = pd.concat(container, axis=1)
# Add current unit to link_ratio
for fnid_new in link_ratio.keys():
    link_ratio[fnid_new][fnid_new] = 1.0
    link_ratio[fnid_new] = link_ratio[fnid_new].sort_index(axis=1, ascending=False)
# Manual Editing
link_ratio['MW2003A20105'][['MW2012R3010501', 'MW2012R3010502']] = 1.0
link_ratio['MW2003A20206'][['MW2014R3020601', 'MW2014R3020602']] = 1.0
# Connect data with AdminLink
area_new, prod_new = FDW_PD_ConnectAdminLink(link_ratio, area, prod, validation=True)
# ----------------------------------------------- #

In [6]:
# Manual correction ----------------------------- #
crop_new = prod_new/area_new
# Potential typo: 567208.0 -> 156720.8
area_new.loc[2016,pd.IndexSlice['MW2003A20206',:,'Maize','Main','04-01','07-01','none']] = 156720.8
# Potential typo: 272.0 -> 2720
prod_new.loc[2016,pd.IndexSlice['MW2003A20311',:,'Maize','Main','04-01','07-01','none']] = 2720
# Potential typo: 96271.0 -> 46271.0
area_new.loc[2004,pd.IndexSlice['MW2003A20307',:,'Maize','Main','04-01','07-01','none']] = 46271
# # ----------------------------------------------- #

# Complete long format DataFrame ---------------- #
df_area = area_new.T.stack().rename('value').reset_index()
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().rename('value').reset_index()
df_prod['indicator'] = 'production'
df_yield = (prod_new/area_new).T.stack().rename('value').reset_index()
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Add "planting year"
cols = ['season_name','product','crop_production_system','planting_month','harvest_year','harvest_month','planting_year']
season_table = df[cols].drop_duplicates()
if FlagGrainType: 
    season_table['product'].replace(product_category,inplace=True)
    season_table = season_table.drop_duplicates()
stack = stack.merge(season_table, on=cols[:-1])
# Add country and admin names
stack = stack.merge(df[['fnid','country','country_code','admin_1','admin_2']].drop_duplicates(), on='fnid', how='inner')
names = [
    'fnid','country','country_code','admin_1','admin_2','name',
    'product','season_name','planting_year','planting_month','harvest_year','harvest_month',
    'crop_production_system','indicator','value'
]
stack_gscd = stack[names]
stack_gscd['gscd_code'] = 'calibrated'
# ----------------------------------------------- #

# Reported FDW data ----------------------------- #
stack_fdw = df[names]
stack_fdw['indicator'] = stack_fdw['indicator'].replace({'Area Harvested':'area','Quantity Produced':'production','Yield':'yield'})
stack_fdw['gscd_code'] = 'reported'
# ----------------------------------------------- #

# Final Processing ------------------------------ #
stack = pd.concat([stack_fdw,stack_gscd], axis=0).reset_index(drop=True)
# No concerns found for grain types
stack['product'] = stack['product'].replace(product_category)
# Calibration of planting and Harvest year and season
cs = {
    'Main': {'planting_month':{'04-01':'11-01'},'harvest_month':{'07-01':'07-01'}},
    'Winter': {'planting_month':{'09-01':'04-01'},'harvest_month':{'11-01':'11-01'}},
}
cy = {'Main': {'planting_year':-1}}
stack, df, link_ratio = FDW_PD_CaliSeasonYear(stack, df, link_ratio, cs, cy)
stack.loc[stack['admin_1'].isna(), 'admin_1'] = 'none'
stack.loc[stack['admin_2'].isna(), 'admin_2'] = 'none'
# ----------------------------------------------- #

# Save data
fn_out = '../data/crop/adm_crop_production_MW.csv'
stack.to_csv(fn_out); print(f'{fn_out} is saved.')
save_hdf('../data/crop/adm_crop_production_MW.hdf', stack)
save_hdf('../data/crop/adm_crop_production_MW_raw.hdf', df)
save_npz('../data/crop/adm_crop_production_MW_ratio.npz', link_ratio)

../data/crop/adm_crop_production_MW.csv is saved.
../data/crop/adm_crop_production_MW.hdf is saved.
../data/crop/adm_crop_production_MW_raw.hdf is saved.
../data/crop/adm_crop_production_MW_ratio.npz is saved.


In [2]:
# Bar chart of national grain production
country_iso, country_name = 'MW', 'Malawi'
df = pd.read_hdf('../data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']

In [3]:
df

,fnid,country,country_code,admin_1,admin_2,name,product,season_name,planting_year,planting_month,harvest_year,harvest_month,crop_production_system,indicator,value,gscd_code,year
10872,MW2003A20101,Malawi,MW,Northern,Chitipa,Chitipa,Maize,Winter,2007,04-01,2007,11-01,none,area,227.000000,calibrated,2007
10873,MW2003A20101,Malawi,MW,Northern,Chitipa,Chitipa,Maize,Winter,2007,04-01,2007,11-01,none,production,251.000000,calibrated,2007
10874,MW2003A20101,Malawi,MW,Northern,Chitipa,Chitipa,Maize,Winter,2007,04-01,2007,11-01,none,yield,1.105727,calibrated,2007
10875,MW2003A20101,Malawi,MW,Northern,Chitipa,Chitipa,Maize,Winter,2008,04-01,2008,11-01,none,area,1143.000000,calibrated,2008
10876,MW2003A20101,Malawi,MW,Northern,Chitipa,Chitipa,Maize,Winter,2008,04-01,2008,11-01,none,production,1950.000000,calibrated,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21462,MW2003A20207,Malawi,MW,Central,Mchinji,Mchinji,Maize,Main,2018,11-01,2019,07-01,estate,yield,3.971295,calibrated,2019
21463,MW2003A20207,Malawi,MW,Central,Mchinji,Mchinji,Maize,Main,2005,11-01,2006,07-01,Estate (PS),area,22226.000000,calibrated,2006
21464,MW2003A20207,Malawi,MW,Central,Mchinji,Mchinji,Maize,Main,2005,11-01,2006,07-01,Estate (PS),production,65866.000000,calibrated,2006
21465,MW2003A20207,Malawi,MW,Central,Mchinji,Mchinji,Maize,Main,2005,11-01,2006,07-01,Estate (PS),yield,2.963466,calibrated,2006


In [8]:
df[
    (df['product']=='Maize') & 
    (df['season_name']=='Main') & 
    (df['indicator']=='production')
].pivot_table(index='year', columns='admin_2', values='value', aggfunc='sum')

admin_2,Balaka,Blantyre,Chikwawa,Chiradzulu,Chitipa,Dedza,Dowa,Karonga,Kasungu,Likoma,...,Nkhata Bay,Nkhotakota,Nsanje,Ntcheu,Ntchisi,Phalombe,Rumphi,Salima,Thyolo,Zomba
year,,,,,,,,,,,,,,,,,,,,,
1984,14558.275483,90350.0,9600.0,NaN,14589.000,191413.000,125143.0,6235.0,74310.0,NaN,...,4404.0,15030.0,2100.000,97356.000,55930.00,39184.0,33560.0,22780.0,NaN,67000.0
1985,18427.268505,91100.0,17200.0,NaN,17647.000,187363.000,113145.0,5221.0,77520.0,NaN,...,4137.0,16068.0,2970.000,87260.000,41000.00,28187.0,31411.0,17364.0,NaN,67665.0
1986,14467.595959,99886.0,18100.0,NaN,12692.000,199846.000,102397.0,7365.0,78000.0,NaN,...,3997.0,12790.0,3140.000,81414.000,36600.00,31593.0,31826.0,26562.0,NaN,52963.0
1987,NaN,108325.0,2982.0,NaN,12028.000,208168.000,95813.0,6761.0,60095.0,NaN,...,4392.0,11220.0,15200.000,89169.000,32200.00,34118.0,23769.0,21480.0,NaN,56980.0
1988,47063.000000,119212.0,23151.0,NaN,13636.000,219835.000,78847.0,5960.0,81252.0,NaN,...,5478.0,3478.0,3422.000,107992.000,35064.00,43807.0,26500.0,25600.0,NaN,68469.0
1989,25906.000000,74116.0,14098.0,NaN,17151.000,175712.000,129670.0,7488.0,94235.0,NaN,...,4055.0,15200.0,1862.000,103723.000,53953.00,41633.0,30288.0,25935.0,NaN,76387.0
1990,29993.000000,83212.0,21219.0,NaN,14735.000,143826.000,83429.0,4916.0,107773.0,NaN,...,2720.0,14784.0,3360.000,88625.000,40496.00,39646.0,35541.0,20261.0,NaN,97369.0
1991,38096.000000,122580.0,31325.0,NaN,14653.000,227210.000,98070.0,6951.0,106651.0,NaN,...,2894.0,14856.0,13678.000,100462.000,68881.00,30670.0,33553.0,27280.0,NaN,99501.0
1992,205.000000,35190.0,3993.0,NaN,7906.000,112386.000,58767.0,1531.0,69797.0,NaN,...,2922.0,6989.0,1318.000,20234.000,37010.00,13752.0,17108.0,6862.0,NaN,20718.0


## Visualization of production data

In [9]:
# Bar chart of national grain production
country_iso, country_name = 'MW', 'Malawi'
df = pd.read_hdf('../data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_order = ['Maize','Millet','Sorghum','Wheat','Rice']
for season_name in ['Main','Winter']:
    footnote = 'National grain production in %s - %s' % (country_name, season_name)
    fn_save = '../figures/%s_bar_natgrainprod_%s.png' % (country_iso, season_name)
    sub = df[df['season_name'] == season_name]
    fig = PlotBarProduction(sub, year, product_order, footnote, fn_save)
    # fig.show()

../figures/MW_bar_natgrainprod_Main.png is saved.
../figures/MW_bar_natgrainprod_Winter.png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/MW_bar_natgrainprod_Main.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/MW_bar_natgrainprod_Winter.png?raw=true)

In [10]:
# Lineplot of Production-Area-Yield (PAY) time-series
country_iso, country_name = 'MW', 'Malawi'
df = pd.read_hdf('../data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_season = [
    ['Maize','Main']
]
for product_name, season_name in product_season:
    footnote = 'Production-Area-Yield (PAY) time-series of %s - %s - %s' % (country_iso, product_name, season_name)
    fn_save = '../figures/%s_line_pay_%s_%s.png' % (country_iso, product_name, season_name)
    sub = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    fig = PlotLinePAY(sub, year, footnote, fn_save)
    # fig.show()

../figures/MW_line_pay_Maize_Main.png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/MW_line_pay_Maize_Main.png?raw=true)

In [11]:
# Heatmap of seasonal data availability
country_iso, country_name = 'MW', 'Malawi'
df = pd.read_hdf('../data/crop/adm_crop_production_%s_raw.hdf' % country_iso)
df['year'] = df['harvest_year']
code = {'Main':1,'Winter':10}
comb = {1:1,10:2,11:3,12:3,20:3,21:3,22:3}
comb_name = {1:'Main',2:'Winter',3:'Main + Winter'}
for product_name in ['Maize Grain (White)']:
    data = df[(df['product'] == product_name) & (df['season_name'].isin(code.keys()))]
    footnote = 'Seasonal data availability in %s - %s (uncorrected)' % (country_name, product_name)
    fn_save = '../figures/%s_heat_seasondata_%s.png' % (country_iso, product_name)
    fig = PlotHeatSeasonData(data, code, comb, comb_name, footnote, fn_save)
    # fig.show()

../figures/MW_heat_seasondata_Maize Grain (White).png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/MW_heat_seasondata_Maize%20Grain%20(White).png?raw=true)

In [12]:
# Heatmap of crop production system(s) per crop type
country_iso, country_name = 'MW', 'Malawi'
df = pd.read_hdf('../data/crop/adm_crop_production_%s_raw.hdf' % country_iso)
df['year'] = df['harvest_year']
product_season = [
    ['Maize Grain (White)', 'Main'],
    ['Maize Grain (White)', 'Winter'],
]
code = {'none':1,'Estate (PS)':10,'estate':100}
comb = {1:1,10:2,11:3,100:4,101:5,110:6,111:7}
comb_name = {1:'None',2:'Estate (PS)',3:'Estate (PS) + None',4:'estate',5:'estate + None',6:'estate + Estate (PS)',7:'All'}
for product_name, season_name in product_season:
    data = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    footnote = 'Reported crop production system(s) in %s - %s - %s (uncorrected)' % ('Somalia', product_name, season_name)
    fn_save = '../figures/%s_heat_cropsystem_%s_%s.png' % (country_iso, product_name, season_name)
    fig = PlotHeatCropSystem(data, code, comb, comb_name, footnote, fn_save)
    # fig.show()

../figures/MW_heat_cropsystem_Maize Grain (White)_Main.png is saved.
../figures/MW_heat_cropsystem_Maize Grain (White)_Winter.png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/MW_heat_cropsystem_Maize%20Grain%20(White)_Main.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/MW_heat_cropsystem_Maize%20Grain%20(White)_Winter.png?raw=true)

In [13]:
# Calibrated PAY time-series per FNID
from tools_graphic import PlotLineCropTS
country_iso, country_name = 'MW', 'Malawi'
df = pd.read_hdf('../data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
link_ratio = load_npz('../data/crop/adm_crop_production_%s_ratio.npz' % country_iso)
year_all = np.arange(df['year'].min(), df['year'].max()+1)
prod_season = [
    ['Maize', 'Main'],
]
for product_name, season_name in prod_season:
    sub = df[
        (df['product'] == product_name) &
        (df['season_name'] == season_name)
    ]
    for fnid in sub['fnid'].unique():
        sub_fps = sub[sub['fnid'] == fnid]
        fn_save = '../figures/crop_calibrated/%s_%s_%s_%s.png' % (country_iso, product_name, season_name, fnid)
        fig = PlotLineCropTS(sub_fps, fnid, product_name, season_name, link_ratio, year_all, fn_save)

../figures/crop_calibrated/MW_Maize_Main_MW2003A20101.png is saved.
../figures/crop_calibrated/MW_Maize_Main_MW2003A20102.png is saved.
../figures/crop_calibrated/MW_Maize_Main_MW2003A20103.png is saved.
../figures/crop_calibrated/MW_Maize_Main_MW2003A20104.png is saved.
../figures/crop_calibrated/MW_Maize_Main_MW2003A20105.png is saved.
../figures/crop_calibrated/MW_Maize_Main_MW2003A20106.png is saved.
../figures/crop_calibrated/MW_Maize_Main_MW2003A20202.png is saved.
../figures/crop_calibrated/MW_Maize_Main_MW2003A20205.png is saved.
../figures/crop_calibrated/MW_Maize_Main_MW2003A20206.png is saved.
../figures/crop_calibrated/MW_Maize_Main_MW2003A20208.png is saved.
../figures/crop_calibrated/MW_Maize_Main_MW2003A20209.png is saved.
../figures/crop_calibrated/MW_Maize_Main_MW2003A20301.png is saved.
../figures/crop_calibrated/MW_Maize_Main_MW2003A20302.png is saved.
../figures/crop_calibrated/MW_Maize_Main_MW2003A20303.png is saved.
../figures/crop_calibrated/MW_Maize_Main_MW2003A